<a href="https://colab.research.google.com/github/lucas-fpaiva/AnalisePred_I/blob/main/Projeto/popularity_forro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Avaliar a popularidade de músicas de forro

In [2]:
import pandas as pd
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import sklearn

from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression

from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
# from sklearn.metrics import d2_pinball_score as pinball
from sklearn.metrics import d2_tweedie_score as tweedie


from sklearn.datasets import make_classification
from ast import literal_eval

import warnings
warnings.filterwarnings('ignore')


In [3]:
def norm_mm(arr):
    return (arr-np.min(arr))/(np.max(arr)-np.min(arr))

def padroniza(data):
    data['artist'] = norm_mm(data['artist_fac'])
    data['track_year'] = norm_mm(data['track_year'])
    data['tempo'] = norm_mm(data['tempo'])
    data['time_signature'] = norm_mm(data['time_signature'])
    data['popularity']=norm_mm(data['popularity'])
    data['key'] = norm_mm(data['key'])
    data['duration_ms'] = norm_mm(data['duration_ms'])
    data['loudness'] = norm_mm(data['loudness'])
  
    return data

In [9]:
def grid_best_model (model="DT", names=False, grid="", task="R"):
  #A função retorna o modelo com os melhores parametros, os melhores parametros e o melhor score
  scoring = 'neg_mean_squared_error'
  #models = ["KNN","LM","DT","SVM", "RF", "MLP", "GB", "ADA"]
  #types = ["C","R"]
  if names == True:
    print("KNN: KNeighborsRegressor\nLM: LinearRegression\nDT: DecisionTreeRegressor\nRF: RandomForestRegressor",
          "\nGB: GradientBoostingRegressor\nSVM: Support Vector Machine\nADA: AdaBoostRegressor\nMLP: MLPRegressor")
    
  if model == "DT":
    if task == "C":
      gs_cv = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=grid['DT'], cv= 5, scoring='accuracy')
      gs_cv.fit(x_treino, y_treino)
      return DecisionTreeClassifier(criterion=gs_cv.best_params_['criterion'], max_depth=gs_cv.best_params_['max_depth'], min_samples_leaf=gs_cv.best_params_['min_samples_leaf'], 
                                   n_estimators=gs_cv.best_params_['n_estimators'], random_state=gs_cv.best_params_['random_state']).fit(x_treino, y_treino), gs_cv.best_params_

    elif task == "R":
      gs_cv = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=grid['DT'], cv= 5, scoring=scoring, n_jobs=-1)
      gs_cv.fit(x_treino, y_treino)
      return DecisionTreeRegressor(criterion=gs_cv.best_params_['criterion'], max_depth=gs_cv.best_params_['max_depth'], min_samples_leaf=gs_cv.best_params_['min_samples_leaf'],
                                   random_state=gs_cv.best_params_['random_state']).fit(x_treino, y_treino), gs_cv.best_params_
        
  if model=="MLP":
    if task == "C":
      gs_cv = GridSearchCV(estimator=MLPClassifier(), param_grid=grid['MLP'], cv= 5, scoring=scoring, n_jobs=-1)
      gs_cv.fit(x_treino, y_treino)
      return MLPClassifier(hidden_layer_sizes=gs_cv.best_params_['hidden_layer_sizes'], max_iter=gs_cv.best_params_['max_iter'], activation=gs_cv.best_params_['activation'], 
                           random_state=gs_cv.best_params_['random_state']), gs_cv.best_params_
    elif task == "R":

      gs_cv = GridSearchCV(estimator=MLPRegressor(), param_grid=grid['MLP'], cv= 5, scoring=scoring, n_jobs=-1)
      gs_cv.fit(x_treino, y_treino)
      return MLPRegressor(early_stopping=True, hidden_layer_sizes=gs_cv.best_params_['hidden_layer_sizes'], max_iter=gs_cv.best_params_['max_iter'], activation=gs_cv.best_params_['activation'], 
                           random_state=gs_cv.best_params_['random_state']), gs_cv.best_params_
    
  if model == "RF":
    if task == "C":
      gs_cv = GridSearchCV(estimator=RandomForestClassifier(), param_grid=grid['RF'], cv= 5, scoring=scoring, n_jobs=-1)
      gs_cv.fit(x_treino, y_treino)
      return RandomForestClassifier(n_estimators=gs_cv.best_params_['n_estimators'], max_depth=gs_cv.best_params_['max_depth'], 
                             random_state=gs_cv.best_params_['random_state'], criterion=gs_cv.best_params_['criterion']), gs_cv.best_params_

    elif task == "R":
      gs_cv = GridSearchCV(estimator=RandomForestRegressor(), param_grid=grid['RF'], cv= 5, scoring=scoring, n_jobs=-1)
      gs_cv.fit(x_treino, y_treino)
      return RandomForestRegressor(n_estimators=gs_cv.best_params_['n_estimators'], max_depth=gs_cv.best_params_['max_depth'], max_features=gs_cv.best_params_['max_features'], 
                             random_state=gs_cv.best_params_['random_state'], criterion=gs_cv.best_params_['criterion']), gs_cv.best_params_

  if model == "KNN":
    if task == "C":
      gs_cv = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=grid['KNN'], cv= 5, scoring=scoring, n_jobs=-1)
      gs_cv.fit(x_treino, y_treino)
      return KNeighborsClassifier(n_neighbors=gs_cv.best_params_['n_neighbors'], weights=gs_cv.best_params_['weights'], metric=gs_cv.best_params_['metric']), gs_cv.best_params_

    elif task == "R":
      gs_cv = GridSearchCV(estimator=KNeighborsRegressor(), param_grid=grid['KNN'], cv= 5, scoring=scoring, n_jobs=-1)
      gs_cv.fit(x_treino, y_treino)
      return KNeighborsRegressor(n_neighbors=gs_cv.best_params_['n_neighbors'], weights=gs_cv.best_params_['weights'], metric=gs_cv.best_params_['metric']), gs_cv.best_params_

  if model == "LM":
    if task == "C":
      gs_cv = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=grid['LM'], cv= 5, scoring=scoring, n_jobs=-1)
      gs_cv.fit(x_treino, y_treino)
      return KNeighborsClassifier(fit_intercept=gs_cv.best_params_['fit_intercept'], penalty=gs_cv.best_params_['penalty'], tol=gs_cv.best_params_['tol']), gs_cv.best_params_

    elif task == "R":
      gs_cv = GridSearchCV(estimator=LinearRegression(), param_grid=grid['LM'], cv= 5, scoring=scoring, n_jobs=-1)
      gs_cv.fit(x_treino, y_treino)
      return LinearRegression(fit_intercept=gs_cv.best_params_['fit_intercept']), gs_cv.best_params_
        
  return print("KNN: KNeighborsRegressor\nLM: LinearRegression\nDT: DecisionTreeRegressor\nRF: RandomForestRegressor",
          "\nGB: GradientBoostingRegressor\nSVM: Support Vector Machine\nADA: AdaBoostRegressor\nMLP: MLPRegressor")
  
#Parâmetros do Grid
grid = {
    'RF':{ 
        'n_estimators': range(5,30,2),
        'max_depth' : range(5,30,2),
        'criterion' :['squared_error', 'absolute_error'],
        'random_state' : [0]},

    'MLP':{
        'hidden_layer_sizes': range(5,30,2),#[(10,), (15,), (20,), (30,)],
        'max_iter': [200],
        'activation' : ['relu', 'tanh'],
        'random_state' : [0]},

    'DT':{
        'max_depth': range(5,30,2),
        'min_samples_leaf': [1,2,3,4,5],
        'criterion' :['squared_error', 'absolute_error'],
        'random_state' : [0]},

    'KNN':{
      'n_neighbors': range(5,30,2),
      'weights': ['uniform', 'distance'],
      'metric' : ['manhattan', 'euclidean']},

    'LM':{
      'fit_intercept' : [True, False]}
}

## Lendo o Dataset

In [5]:
forroset = pd.read_csv('https://raw.githubusercontent.com/lucas-fpaiva/AnalisePred_I/main/Projeto/new_forroset.csv')
forroset.columns

Index(['Unnamed: 0', 'track', 'track_id', 'artist', 'artist_id', 'popularity',
       'album', 'album_id', 'track_year', 'duration_ms', 'uri', 'preview_url',
       'energy', 'liveness', 'tempo', 'speechiness', 'acousticness',
       'instrumentalness', 'time_signature', 'danceability', 'key', 'loudness',
       'valence', 'mode', 'artist_fac', 'fold', 'popularity_class'],
      dtype='object')

In [6]:
forroset = forroset[["track_id",'popularity', 'duration_ms','track_year', 'artist_fac','energy',
       'liveness', 'tempo', 'speechiness', 'acousticness', 'instrumentalness',
       'time_signature', 'danceability', 'key', 'loudness', 'valence', 'mode', 'fold']]

## Avaliando a Árvore de Decisão

In [7]:
def metrics (y_true, y_pred):
  return [mae(y_true, y_pred), mape(y_true, y_pred), mse(y_true, y_pred), mse(y_true, y_pred, squared=False), r2(y_true, y_pred), tweedie(y_true, y_pred)]

In [10]:
data_results = pd.DataFrame([],columns=['fold','model','parameters','mae','mape', 'mse','rmse','r2','tweedie'])
#models = ["LM", "KNN", "MLP", "DT", "RF" ]
models = ["RF" ]

for name_model in models:
  results = []
  for k in range(1,6):
    treino = forroset[forroset.fold!=k][forroset.popularity>0]
    teste  = forroset[forroset.fold==k][forroset.popularity>0]

    teste = padroniza(teste)
    treino = padroniza(treino)
    
    x_treino = treino.drop(['fold','track_id','popularity'], axis=1)
    y_treino = treino.popularity

    x_teste = teste.drop(['fold','track_id','popularity'], axis=1)
    y_teste = teste.popularity

    model, best_parameters = grid_best_model(model=name_model, grid=grid)
    model.fit(x_treino, y_treino)
    y_pred= model.predict(x_teste)

    print("Melhor parametro:", best_parameters)         
    print("Fold", k, metrics(y_teste, y_pred))

    result = metrics(y_teste, y_pred)
    result.append(k)
    result.append(name_model)
    result.append(best_parameters)
    
    results.append(result)
        
  data_results = pd.concat([data_results, pd.DataFrame(results,columns=['mae','mape', 'mse','rmse','r2','tweedie', 'fold','model','parameters'])])
  data_results.to_csv('/content/drive/MyDrive/AnalisePred_I/results_'+ name_model + '.csv')


KeyboardInterrupt: ignored

In [ ]:
print(data_results['parameters'][4])

In [ ]:
data_results.mean()

### Predição do Modelo

In [ ]:
model.fit(x_treino, y_treino)
np.sqrt(model.score(x_treino, y_treino))

In [ ]:
model.score(x_teste, y_teste)

In [ ]:
y_pred_t = model.predict(x_treino)

In [ ]:
y_pred= model.predict(x_teste)

In [ ]:
plt.plot(y_treino, y_pred_t, 'o')

plt.plot(y_teste, y_pred, 'x')


#### MAPE

In [ ]:
mape(y_teste,y_pred)*100

In [ ]:
plt.plot(((y_teste.values - y_pred)/y_teste.values)*100)
np.mean(100*np.abs(y_teste.values - y_pred)/y_teste.values)

#### Plot do Modelo Teste x Treino

In [ ]:
plt.plot(y_teste.values, 'o')
plt.plot(y_pred, 'x')
plt.xlim([100,300])
plt.ylim([0,0.3])

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / (y_true+0.00001))) * 100

In [ ]:
def mae (y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

In [ ]:
def r2_score(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))